In [1]:
!nvidia-smi

Tue Jul 30 06:01:39 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:00:05.0 Off |                  Off |
| 30%   26C    P8    18W / 300W |      1MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 21.8 MB/s eta 0:00:00


In [37]:
from xgboost import XGBRegressor 
import optuna
import pandas as pd 
import numpy as np 
import os 
import math

In [38]:
train = pd.read_csv("train.csv") 
train.head() 

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [39]:
train_calendar = pd.read_csv("train_calendar.csv") 

train_calendar.head() 

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse
0,2022-03-09,NaN,0,0,0,0,0,1,0,0,0.0,0,0.0,0.0,Prague_1
1,2019-07-03,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
2,2020-05-18,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
3,2021-11-09,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
4,2023-08-02,NaN,0,0,0,0,0,0,0,0,0.0,0,2.5,0.0,Prague_1


In [40]:
train.shape, train_calendar.shape

((7340, 18), (13307, 15))

In [41]:
test = pd.read_csv("test.csv") 
test_calendar = pd.read_csv("test_calendar.csv") 

test.shape, test_calendar.shape

((397, 8), (1869, 7))

In [42]:
test.head() 

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


In [43]:
test_calendar.head() 

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2024-11-17,Den boje za svobodu a demokracii,1,0,0,0,Prague_1
1,2024-07-04,NaN,0,0,0,0,Prague_1
2,2024-08-04,NaN,0,0,0,0,Prague_1
3,2024-12-01,NaN,0,0,0,0,Prague_1
4,2024-07-24,NaN,0,0,0,0,Prague_1


## Preprocess Data

In [44]:
# Merge the train and test datasets with their respective calendar datasets
train = pd.merge(train, train_calendar, on=["date", "warehouse"], how="left", suffixes=('', '_cal'))
test = pd.merge(test, test_calendar, on=["date", "warehouse"], how="left", suffixes=('', '_cal'))

In [45]:
columns_to_drop = ['holiday_name_cal', 'holiday_cal', 'shops_closed_cal', 
                   'winter_school_holidays_cal', 'school_holidays_cal']
train.drop(columns=columns_to_drop, inplace=True)
test.drop(columns=columns_to_drop, inplace=True)

In [46]:
# Align columns: Ensure both train and test datasets have the same columns
# Identify columns in the train set that are not in the test set
columns_train_not_in_test = set(train.columns) - set(test.columns)
# Remove 'orders' and 'id' from this set since they are target and identifier
columns_train_not_in_test = {col for col in columns_train_not_in_test if col not in ['orders', 'id']}

columns_train_not_in_test

{'blackout',
 'blackout_cal',
 'frankfurt_shutdown',
 'frankfurt_shutdown_cal',
 'mini_shutdown',
 'mini_shutdown_cal',
 'mov_change',
 'mov_change_cal',
 'precipitation',
 'precipitation_cal',
 'shutdown',
 'shutdown_cal',
 'snow',
 'snow_cal',
 'user_activity_1',
 'user_activity_2',
 'warehouse_limited'}

In [47]:
# Drop these columns from the train dataset
train.drop(columns=columns_train_not_in_test, axis=1, inplace=True)


In [48]:
train

,warehouse,date,orders,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,Prague_1_2020-12-09
...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,6733.0,NaN,0,0,0,0,Budapest_1_2024-03-10
7336,Budapest_1,2024-03-11,6492.0,NaN,0,0,0,0,Budapest_1_2024-03-11
7337,Budapest_1,2024-03-12,6661.0,NaN,0,0,0,0,Budapest_1_2024-03-12
7338,Budapest_1,2024-03-13,6843.0,NaN,0,0,0,0,Budapest_1_2024-03-13


In [49]:
test

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20
...,...,...,...,...,...,...,...,...
392,Budapest_1,2024-05-11,NaN,0,0,0,0,Budapest_1_2024-05-11
393,Budapest_1,2024-05-12,NaN,0,0,0,0,Budapest_1_2024-05-12
394,Budapest_1,2024-05-13,NaN,0,0,0,0,Budapest_1_2024-05-13
395,Budapest_1,2024-05-14,NaN,0,0,0,0,Budapest_1_2024-05-14


In [50]:
# preprocess date information 
train["date"] = pd.to_datetime(train["date"]) 
train["month"] = train["date"].dt.month 
train["day"] = train["date"].dt.day 
train["weekday"] = train["date"].dt.weekday 

test["date"] = pd.to_datetime(test["date"]) 
test["month"] = test["date"].dt.month 
test["day"] = test["date"].dt.day 
test["weekday"] = test["date"].dt.weekday 

In [51]:
train

,warehouse,date,orders,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,month,day,weekday
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,Prague_1_2020-12-05,12,5,5
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,Prague_1_2020-12-06,12,6,6
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,Prague_1_2020-12-07,12,7,0
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,Prague_1_2020-12-08,12,8,1
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,Prague_1_2020-12-09,12,9,2
...,...,...,...,...,...,...,...,...,...,...,...,...
7335,Budapest_1,2024-03-10,6733.0,NaN,0,0,0,0,Budapest_1_2024-03-10,3,10,6
7336,Budapest_1,2024-03-11,6492.0,NaN,0,0,0,0,Budapest_1_2024-03-11,3,11,0
7337,Budapest_1,2024-03-12,6661.0,NaN,0,0,0,0,Budapest_1_2024-03-12,3,12,1
7338,Budapest_1,2024-03-13,6843.0,NaN,0,0,0,0,Budapest_1_2024-03-13,3,13,2


In [52]:
test

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id,month,day,weekday
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16,3,16,5
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17,3,17,6
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18,3,18,0
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19,3,19,1
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20,3,20,2
...,...,...,...,...,...,...,...,...,...,...,...
392,Budapest_1,2024-05-11,NaN,0,0,0,0,Budapest_1_2024-05-11,5,11,5
393,Budapest_1,2024-05-12,NaN,0,0,0,0,Budapest_1_2024-05-12,5,12,6
394,Budapest_1,2024-05-13,NaN,0,0,0,0,Budapest_1_2024-05-13,5,13,0
395,Budapest_1,2024-05-14,NaN,0,0,0,0,Budapest_1_2024-05-14,5,14,1


In [53]:
# drop unnecessary columns 
train.drop(columns={"date","id"}, inplace=True)  
test.drop(columns={"date", "id"}, inplace=True) 

In [54]:
train.head(2)

,warehouse,orders,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,month,day,weekday
0,Prague_1,6895.0,NaN,0,0,0,0,12,5,5
1,Prague_1,6584.0,NaN,0,0,0,0,12,6,6


In [55]:
test.head(2)

,warehouse,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,month,day,weekday
0,Prague_1,NaN,0,0,0,0,3,16,5
1,Prague_1,NaN,0,0,0,0,3,17,6


In [56]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train['holiday_name'] = label_encoder.fit_transform(train['holiday_name'].fillna('unknown'))

test['holiday_name'] = label_encoder.transform(test['holiday_name'].fillna('unknown'))


In [57]:
np.unique(train["warehouse"]) 

array(['Brno_1', 'Budapest_1', 'Frankfurt_1', 'Munich_1', 'Prague_1',
       'Prague_2', 'Prague_3'], dtype=object)

In [62]:
import xgboost as xgb

train_columns = ["holiday_name", "holiday", "shops_closed", "winter_school_holidays", "school_holidays", "month", "day", "weekday"]
target_columns = ["orders"] 

warehouses = ["Prague_1", "Brno_1", "Prague_2", "Prague_3", "Munich_1", "Frankfurt_1", "Budapest_1"]

predictions = [] 

for warehouse in warehouses: 
    print(f"========= predicting on ware house {warehouse} =========")
    cur_train = train[train["warehouse"]==warehouse] 
    cur_train.drop(columns={"warehouse"}, inplace=True) 
    cur_X = cur_train[train_columns] 
    cur_Y = cur_train[target_columns] 
    
    # Train the XGBoost model with GPU support
    model = xgb.XGBRegressor(objective='reg:squarederror', 
                             n_estimators=100, 
                             learning_rate=0.1, 
                             max_depth=6, 
                             tree_method='gpu_hist')
    model.fit(cur_X, cur_Y)
        
    cur_test = test[test["warehouse"]==warehouse]  
    cur_test.drop(columns={"warehouse"}, inplace=True) 
    
    # Make predictions
    cur_predictions = model.predict(cur_test) 
    
    predictions.append(cur_predictions) 
    

========= predicting on ware house Prague_1 =========
========= predicting on ware house Brno_1 =========


/tmp/ipykernel_78/1884203333.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_train.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_test.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_train.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:26: Se

========= predicting on ware house Prague_2 =========
========= predicting on ware house Prague_3 =========


/tmp/ipykernel_78/1884203333.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_test.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_train.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_test.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:13: Set

========= predicting on ware house Munich_1 =========
========= predicting on ware house Frankfurt_1 =========
========= predicting on ware house Budapest_1 =========


/tmp/ipykernel_78/1884203333.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_train.drop(columns={"warehouse"}, inplace=True)
/tmp/ipykernel_78/1884203333.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_test.drop(columns={"warehouse"}, inplace=True)


In [63]:
len(predictions)

7

In [66]:
full_answer = np.concatenate(predictions) 

In [67]:
len(full_answer)

397

In [70]:
submission = pd.read_csv("solution_example.csv") 

submission["orders"] = full_answer 

In [71]:
submission

,id,orders
0,Prague_1_2024-03-16,8716.627930
1,Prague_1_2024-03-17,8708.703125
2,Prague_1_2024-03-18,8715.503906
3,Prague_1_2024-03-19,8422.078125
4,Prague_1_2024-03-20,8405.274414
...,...,...
392,Budapest_1_2024-05-11,5440.345215
393,Budapest_1_2024-05-12,5019.130371
394,Budapest_1_2024-05-13,5304.162109
395,Budapest_1_2024-05-14,5381.133789


In [72]:
submission.to_csv("xgboost_baseline.csv", index=False) 

print("done!") 

done!
